In [5]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

In [6]:
path_position_df = pd.read_csv('data/drone_path_position.csv')
path_position_df['dev0'] = [np.array(row[1:-1].split()).astype(float) for row in path_position_df['dev0']]
path_position_df['dev1'] = [np.array(row[1:-1].split()).astype(float) for row in path_position_df['dev1']]
path_position_df[:5]

,name,dev0,dev1
0,Image_w1920_h1200_fn12.jpg,"[1223.5, 576.0]","[1421.0, 697.0]"
1,Image_w1920_h1200_fn13.jpg,"[1165.0, 551.5]","[1458.0, 681.0]"
2,Image_w1920_h1200_fn14.jpg,"[1103.5, 520.0]","[1499.5, 663.0]"
3,Image_w1920_h1200_fn15.jpg,"[1043.0, 497.0]","[1546.5, 642.0]"
4,Image_w1920_h1200_fn16.jpg,"[981.5, 474.0]","[1597.0, 628.5]"


In [3]:
# Create a figure
fig = plt.figure(figsize=(10, 10))

# Create subplots
ax0 = fig.add_subplot(1, 2, 1)
ax1 = fig.add_subplot(1, 2, 2)



# Get number of frames
N = len(path_position_df)

# Animation update function
def update(t):
    # Clear each subplot
    ax0.cla()
    ax1.cla()

    # Retrieve the image name and coordinates
    row = path_position_df.iloc[t]
    image_name = row['name']
    dev0_coordinates = row['dev0']
    dev1_coordinates = row['dev1']

    # Load the images
    dev0_image = cv2.imread(f'data/drone_dataset/dev0/Dev0_{image_name}')
    dev1_image = cv2.imread(f'data/drone_dataset/dev1/Dev1_{image_name}')

    # Draw circles on the images
    cv2.circle(dev0_image, (int(dev0_coordinates[0]), int(dev0_coordinates[1])), 40, (0, 255, 0), 3)
    cv2.circle(dev1_image, (int(dev1_coordinates[0]), int(dev1_coordinates[1])), 40, (0, 255, 0), 3)

    # Convert BGR image to RGB for plotting
    dev0_image_rgb = cv2.cvtColor(dev0_image, cv2.COLOR_BGR2RGB)
    dev1_image_rgb = cv2.cvtColor(dev1_image, cv2.COLOR_BGR2RGB)

    # Display dev0 image
    ax0.imshow(dev1_image_rgb)
    ax0.set_title('Dev1 Image')
    ax0.axis('off')

    # Display dev1 image
    ax1.imshow(dev0_image_rgb)
    ax1.set_title('Dev0 Image')
    ax1.axis('off')


# Create animation
ani = animation.FuncAnimation(fig, update, frames=N)

# Display animation
plt.show()


# Triangulate Points to Get World Coordinates

In [7]:
from scipy.io import loadmat


K1 = loadmat('data/camParams/K1.mat')['K1']
K2 = loadmat('data/camParams/K2.mat')['K2']
R2 = loadmat('data/camParams/R2.mat')['R2']
T2 = loadmat('data/camParams/T2.mat')['T2']

### Calculate P1 and P2 projection matrices using Intrinsic and Extrinsic Parameters

In [8]:
P1 = K1 @ np.hstack((np.identity(3), np.zeros((3, 1))))
P2 = K2 @ np.hstack((R2, T2.reshape(3, 1)))

### Write a function that given P1 and P2 and Camera Coordinates of point1 and point2 calculates world coordinates

In [9]:
def triangulatePoints(P1, P2, point1, point2):
    A = np.zeros((4,4))
    for i in range(len(point1)):
        A[i, :] = point1[i]*P1[2, :] - P1[i, :]
        A[i+2, :] = point2[i]*P2[2, :] - P2[i, :]

    _, _, V = np.linalg.svd(A)
    X = V[-1, :4]
    X = X/X[3]
    return X

In [10]:
traj3D = []
for idx, row in path_position_df.iterrows():
    point1 = row['dev0']
    point2 = row['dev1']
    x, y, z, _ = triangulatePoints(P1, P2, point1.T, point2.T)


    traj3D.append([x,y,z])

#### Visualise the result as animation

In [ ]:
traj3D = np.array(traj3D) / 1000
# Create a figure
fig = plt.figure(figsize=(20, 10))

# Create subplots
ax0 = fig.add_subplot(1, 3, 1)
ax1 = fig.add_subplot(1, 3, 2)
ax2 = fig.add_subplot(1, 3, 3, projection='3d')

# Set 3D plot title and labels
ax2.set_title('3D Trajectory of the Drone')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')

# Get number of frames
N = len(path_position_df)

# Animation update function
def update(t):
    # Clear each subplot
    ax0.cla()
    ax1.cla()
    ax2.cla()

    # Retrieve the image name and coordinates
    row = path_position_df.iloc[t]
    image_name = row['name']
    dev0_coordinates = row['dev0']
    dev1_coordinates = row['dev1']

    # Load the images
    dev0_image = cv2.imread(f'data/drone_dataset/dev0/Dev0_{image_name}')
    dev1_image = cv2.imread(f'data/drone_dataset/dev1/Dev1_{image_name}')

    # Draw circles on the images
    cv2.circle(dev0_image, (int(dev0_coordinates[0]), int(dev0_coordinates[1])), 40, (0, 255, 0), 3)
    cv2.circle(dev1_image, (int(dev1_coordinates[0]), int(dev1_coordinates[1])), 40, (0, 255, 0), 3)

    # Convert BGR image to RGB for plotting
    dev0_image_rgb = cv2.cvtColor(dev0_image, cv2.COLOR_BGR2RGB)
    dev1_image_rgb = cv2.cvtColor(dev1_image, cv2.COLOR_BGR2RGB)

    # Display dev0 image
    ax0.imshow(dev1_image_rgb)
    ax0.set_title('Dev1 Image')
    ax0.axis('off')

    # Display dev1 image
    ax1.imshow(dev0_image_rgb)
    ax1.set_title('Dev0 Image')
    ax1.axis('off')

    # Display 3D trajectory
    ax2.plot(traj3D[:t+1, 0], traj3D[:t+1, 1], traj3D[:t+1, 2], color='b')
    ax2.set_title('3D Trajectory of the Drone')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')

# Create animation
ani = animation.FuncAnimation(fig, update, frames=N)

# Display animation
plt.show()


# Validate Result

In [9]:
rmse = lambda a,b: np.sqrt((a - b).flatten() @ (a - b).flatten())

if rmse(traj3D, np.loadtxt('data/drone_3Dtraj.txt')) < 0.0001:
    print('error',rmse(traj3D, np.loadtxt('data/drone_3Dtraj.txt')), ' | passed')

error 3.550600581939424e-11  | passed
